In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# Show all output for a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [36]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [37]:
from src.inference import get_feature_store

In [38]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*40)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.hour.between(fetch_data_from, fetch_data_to)]

2025-05-11 10:16:44,480 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 10:16:44,505 INFO: Initializing external client
2025-05-11 10:16:44,506 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:16:45,124 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-01 14:16:44.479502+00:00 to 2025-05-11 13:16:44.479502+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 


In [39]:
ts_data.sort_values(["start_station_id", "hour"]).reset_index(drop=True)

,hour,start_station_id,ride_count
0,2025-04-12 10:00:00+00:00,5329.03,12
1,2025-04-12 11:00:00+00:00,5329.03,22
2,2025-04-12 12:00:00+00:00,5329.03,18
3,2025-04-12 13:00:00+00:00,5329.03,18
4,2025-04-12 14:00:00+00:00,5329.03,13
...,...,...,...
2095,2025-05-11 09:00:00+00:00,6140.05,3
2096,2025-05-11 10:00:00+00:00,6140.05,17
2097,2025-05-11 11:00:00+00:00,6140.05,16
2098,2025-05-11 12:00:00+00:00,6140.05,7


In [40]:
ts_data["hour"] = ts_data["hour"].dt.tz_localize(None)

In [41]:
from src.data_utils import transform_ts_data_into_features
features = transform_ts_data_into_features(ts_data, window_size=24*28, step_size=23)

In [42]:
features.head()

,ride_count_t-672,ride_count_t-671,ride_count_t-670,ride_count_t-669,ride_count_t-668,ride_count_t-667,ride_count_t-666,ride_count_t-665,ride_count_t-664,ride_count_t-663,...,ride_count_t-8,ride_count_t-7,ride_count_t-6,ride_count_t-5,ride_count_t-4,ride_count_t-3,ride_count_t-2,ride_count_t-1,start_station_id,hour
0,20,0,0,24,8,4,0,11,18,27,...,35,37,25,8,6,16,27,1,5905.14,2025-05-05 05:00:00
1,1,4,4,0,7,52,16,9,49,19,...,37,2,0,9,0,3,32,16,5905.14,2025-04-25 08:00:00
2,73,1,0,27,4,0,0,21,0,16,...,22,13,13,11,19,29,0,22,5329.03,2025-04-15 13:00:00
3,28,21,38,4,2,0,2,10,21,0,...,48,100,10,6,21,17,10,83,5329.03,2025-05-08 18:00:00
4,27,35,17,41,42,6,28,35,35,4,...,24,51,25,32,9,35,2,1,6140.05,2025-05-11 07:00:00


In [43]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-11 10:16:49,325 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 10:16:49,331 INFO: Initializing external client
2025-05-11 10:16:49,332 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:16:49,819 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-12 14:16:49.325597+00:00 to 2025-05-11 13:16:49.325597+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 


In [44]:
features.shape

(6, 674)

In [45]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-11 10:16:52,982 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 10:16:52,987 INFO: Initializing external client
2025-05-11 10:16:52,988 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:16:53,493 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


Downloading: 0.000%|          | 0/324120 elapsed<00:00 remaining<?

In [46]:
features.shape

(6, 674)

In [47]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [48]:
predictions

,start_station_id,predicted_demand
0,5329.03,39.0
1,5329.03,27.0
2,5905.14,0.0
3,5905.14,19.0
4,6140.05,33.0
5,6140.05,29.0
